In [408]:
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import json

In [409]:
data_df = pd.read_csv('./src/api/data/train_ML_IOT.csv', sep = ",", on_bad_lines='skip')

In [410]:
data_df

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041
...,...,...,...,...
48115,2017-06-30 19:00:00,4,11,20170630194
48116,2017-06-30 20:00:00,4,30,20170630204
48117,2017-06-30 21:00:00,4,16,20170630214
48118,2017-06-30 22:00:00,4,22,20170630224


In [411]:
grouped = data_df.groupby('DateTime')

In [412]:
grouped.head()

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041
...,...,...,...,...
48115,2017-06-30 19:00:00,4,11,20170630194
48116,2017-06-30 20:00:00,4,30,20170630204
48117,2017-06-30 21:00:00,4,16,20170630214
48118,2017-06-30 22:00:00,4,22,20170630224


In [413]:
data_df = data_df[data_df["Junction"] == 1]

In [414]:
data_df

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041
...,...,...,...,...
14587,2017-06-30 19:00:00,1,105,20170630191
14588,2017-06-30 20:00:00,1,96,20170630201
14589,2017-06-30 21:00:00,1,90,20170630211
14590,2017-06-30 22:00:00,1,84,20170630221


In [415]:
data_df.isna().sum()

DateTime    0
Junction    0
Vehicles    0
ID          0
dtype: int64

In [416]:
data_df["DateTime"] = pd.to_datetime(data_df["DateTime"])
d = data_df["DateTime"] 
data_df.index = pd.DatetimeIndex(d) #.sort_index(ascending=True, inplace=False)


/var/folders/xk/ct1qcm4j5ljdj441cx4jphvjp9l86x/T/ipykernel_52852/2952992933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["DateTime"] = pd.to_datetime(data_df["DateTime"])


In [417]:
data_df = data_df[:10000]

In [418]:
data_df

,DateTime,Junction,Vehicles,ID
DateTime,,,,
2015-11-01 00:00:00,2015-11-01 00:00:00,1,15,20151101001
2015-11-01 01:00:00,2015-11-01 01:00:00,1,13,20151101011
2015-11-01 02:00:00,2015-11-01 02:00:00,1,10,20151101021
2015-11-01 03:00:00,2015-11-01 03:00:00,1,7,20151101031
2015-11-01 04:00:00,2015-11-01 04:00:00,1,9,20151101041
...,...,...,...,...
2016-12-21 11:00:00,2016-12-21 11:00:00,1,76,20161221111
2016-12-21 12:00:00,2016-12-21 12:00:00,1,82,20161221121
2016-12-21 13:00:00,2016-12-21 13:00:00,1,76,20161221131


In [419]:
# data_df.index = data_df.index + pd.DateOffset(years=3, days=17, months=1) - pd.DateOffset(hours=6)
data_df["DateTime"] = data_df.index

/var/folders/xk/ct1qcm4j5ljdj441cx4jphvjp9l86x/T/ipykernel_52852/4128591110.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["DateTime"] = data_df.index


In [420]:
data_df = data_df.drop(columns=['Junction'])

In [421]:
data_df = data_df.drop(columns=['ID'])
data_df = data_df.rename(columns={"DateTime": "timestamp"})


In [422]:
data_df = data_df.rename(columns={"Vehicles": "vehicle count"})

In [423]:
data_df

,timestamp,vehicle count
DateTime,,
2015-11-01 00:00:00,2015-11-01 00:00:00,15
2015-11-01 01:00:00,2015-11-01 01:00:00,13
2015-11-01 02:00:00,2015-11-01 02:00:00,10
2015-11-01 03:00:00,2015-11-01 03:00:00,7
2015-11-01 04:00:00,2015-11-01 04:00:00,9
...,...,...
2016-12-21 11:00:00,2016-12-21 11:00:00,76
2016-12-21 12:00:00,2016-12-21 12:00:00,82
2016-12-21 13:00:00,2016-12-21 13:00:00,76


In [424]:
data_df["vehicle count"] = data_df["vehicle count"] + 51 + np.random.randint(1, 5, size=len(data_df)) 


In [425]:
data_df[data_df["vehicle count"] > 262]

,timestamp,vehicle count
DateTime,,


In [426]:
# data_df["is_night"] = (data_df["timestamp"].dt.hour > 23) & (data_df["timestamp"].dt.hour < 5)
# data_df["is_peak"] = ((data_df["timestamp"].dt.hour > 6) & (data_df["timestamp"].dt.hour < 10)) | ((data_df["timestamp"].dt.hour > 16) & (data_df["timestamp"].dt.hour < 20))

# data_df["vehicle count"] = np.where(data_df["is_peak"], (data_df["vehicle count"] + 100), data_df["vehicle count"] - 200)
# data_df["vehicle count"] = np.where(data_df["is_night"], (data_df["vehicle count"] / 4), data_df["vehicle count"])
# data_df["vehicle count"] = np.where(data_df["timestamp"].dt.hour == 10, (data_df["vehicle count"] - 200), data_df["vehicle count"])


In [427]:
pd.to_datetime('2020-01-16T17:00')

Timestamp('2020-01-16 17:00:00')

In [428]:
# data_df = pd.concat([data_df, 
#                      pd.DataFrame({
#     'vehicle count': [640], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 17:00:00')
# }, index=[pd.to_datetime('2020-01-16 17:00:00')]),
#                      pd.DataFrame({
#     'vehicle count': [680], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 18:00:00')
# }, index=[pd.to_datetime('2020-01-16 18:00')]),
# pd.DataFrame({
#     'vehicle count': [690], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 19:00:00')
# }, index=[pd.to_datetime('2020-01-16 19:00')]),
# pd.DataFrame({
#     'vehicle count': [840], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 20:00:00')
# }, index=[pd.to_datetime('2020-01-16 20:00')]),
# pd.DataFrame({
#     'vehicle count': [790], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 21:00:00')
# }, index=[pd.to_datetime('2020-01-16 21:00')]),
# pd.DataFrame({
#     'vehicle count': [760], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 22:00:00')
# }, index=[pd.to_datetime('2020-01-16 22:00')]),
# pd.DataFrame({
#     'vehicle count': [460], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 23:00:00')
# }, index=[pd.to_datetime('2020-01-16 23:00')]),
# pd.DataFrame({
#     'vehicle count': [320], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 00:00:00')
# }, index=[pd.to_datetime('2020-01-17 00:00')]),
# pd.DataFrame({
#     'vehicle count': [330], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 01:00:00')
# }, index=[pd.to_datetime('2020-01-17 01:00')]),
# pd.DataFrame({
#     'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 02:00:00')
# }, index=[pd.to_datetime('2020-01-17 03:00')]),
# pd.DataFrame({
#     'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 03:00:00')
# }, index=[pd.to_datetime('2020-01-17 03:00')]),
# pd.DataFrame({
#     'vehicle count': [360], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 04:00:00')
#   }, index=[pd.to_datetime('2020-01-17 04:00')]),
#     pd.DataFrame({
#     'vehicle count': [370], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 05:00:00')
#   }, index=[pd.to_datetime('2020-01-17 05:00')]),
#       pd.DataFrame({
#     'vehicle count': [590], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 06:00:00')
#   }, index=[pd.to_datetime('2020-01-17 06:00')]),
#        pd.DataFrame({
#     'vehicle count': [980], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 07:00:00')
#   }, index=[pd.to_datetime('2020-01-17 07:00')]),
#          pd.DataFrame({
#     'vehicle count': [967], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 08:00:00')
#   }, index=[pd.to_datetime('2020-01-17 08:00')]),
#            pd.DataFrame({
#     'vehicle count': [767], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 09:00:00')
#   }, index=[pd.to_datetime('2020-01-17 09:00')]),
#              pd.DataFrame({
#     'vehicle count': [617], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 10:00:00')
#   }, index=[pd.to_datetime('2020-01-17 10:00')]),
#              pd.DataFrame({
#     'vehicle count': [587], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 11:00:00')
#   }, index=[pd.to_datetime('2020-01-17 11:00')]),
#                pd.DataFrame({
#     'vehicle count': [577], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 12:00:00')
#   }, index=[pd.to_datetime('2020-01-17 12:00')]),
#                  pd.DataFrame({
#     'vehicle count': [687], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 13:00:00')
#   }, index=[pd.to_datetime('2020-01-17 13:00')]),
#                    pd.DataFrame({
#     'vehicle count': [657], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 14:00:00')
#   }, index=[pd.to_datetime('2020-01-17 14:00')]),
#                    pd.DataFrame({
#     'vehicle count': [657], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 15:00:00')
#   }, index=[pd.to_datetime('2020-01-17 15:00')]),
#                      pd.DataFrame({
#     'vehicle count': [627], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 16:00:00')
#   }, index=[pd.to_datetime('2020-01-17 16:00')]),
#                        pd.DataFrame({
#     'vehicle count': [857], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 17:00:00')
#   }, index=[pd.to_datetime('2020-01-17 17:00')]),
#                          pd.DataFrame({
#     'vehicle count': [867], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 18:00:00')
#   }, index=[pd.to_datetime('2020-01-17 18:00')])
# ])

In [429]:

# data_df = pd.concat([data_df, 
#                      pd.DataFrame({
#     'vehicle count': [640], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 17:00:00')
# }, index=[pd.to_datetime('2019-11-16 17:00:00')]),
#                      pd.DataFrame({
#     'vehicle count': [680], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 18:00:00')
# }, index=[pd.to_datetime('2019-11-16 18:00')]),
# pd.DataFrame({
#     'vehicle count': [690], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 19:00:00')
# }, index=[pd.to_datetime('2019-11-16 19:00')]),
# pd.DataFrame({
#     'vehicle count': [840], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 20:00:00')
# }, index=[pd.to_datetime('2019-11-16 20:00')]),
# pd.DataFrame({
#     'vehicle count': [790], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 21:00:00')
# }, index=[pd.to_datetime('2019-11-16 21:00')]),
# pd.DataFrame({
#     'vehicle count': [760], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 22:00:00')
# }, index=[pd.to_datetime('2019-11-16 22:00')]),
# pd.DataFrame({
#     'vehicle count': [460], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 23:00:00')
# }, index=[pd.to_datetime('2019-11-16 23:00')]),
# pd.DataFrame({
#     'vehicle count': [320], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 00:00:00')
# }, index=[pd.to_datetime('2019-11-17 00:00')]),
# pd.DataFrame({
#     'vehicle count': [330], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 01:00:00')
# }, index=[pd.to_datetime('2019-11-17 01:00')]),
# pd.DataFrame({
#     'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 02:00:00')
# }, index=[pd.to_datetime('2019-11-17 03:00')]),
# pd.DataFrame({
#     'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 03:00:00')
# }, index=[pd.to_datetime('2019-11-17 03:00')]),
# pd.DataFrame({
#     'vehicle count': [360], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 04:00:00')
#   }, index=[pd.to_datetime('2019-11-17 04:00')]),
#     pd.DataFrame({
#     'vehicle count': [370], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 05:00:00')
#   }, index=[pd.to_datetime('2019-11-17 05:00')]),
#       pd.DataFrame({
#     'vehicle count': [590], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 06:00:00')
#   }, index=[pd.to_datetime('2019-11-17 06:00')]),
#        pd.DataFrame({
#     'vehicle count': [980], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 07:00:00')
#   }, index=[pd.to_datetime('2019-11-17 07:00')]),
#          pd.DataFrame({
#     'vehicle count': [967], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 08:00:00')
#   }, index=[pd.to_datetime('2019-11-17 08:00')]),
#            pd.DataFrame({
#     'vehicle count': [767], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 09:00:00')
#   }, index=[pd.to_datetime('2019-11-17 09:00')]),
#              pd.DataFrame({
#     'vehicle count': [617], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 10:00:00')
#   }, index=[pd.to_datetime('2019-11-17 10:00')]),
#              pd.DataFrame({
#     'vehicle count': [587], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 11:00:00')
#   }, index=[pd.to_datetime('2019-11-17 11:00')]),
#                pd.DataFrame({
#     'vehicle count': [577], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 12:00:00')
#   }, index=[pd.to_datetime('2019-11-17 12:00')]),
#                  pd.DataFrame({
#     'vehicle count': [687], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 13:00:00')
#   }, index=[pd.to_datetime('2019-11-17 13:00')]),
#                    pd.DataFrame({
#     'vehicle count': [657], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 14:00:00')
#   }, index=[pd.to_datetime('2019-11-17 14:00')]),
#                    pd.DataFrame({
#     'vehicle count': [657], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 15:00:00')
#   }, index=[pd.to_datetime('2019-11-17 15:00')]),
#                      pd.DataFrame({
#     'vehicle count': [627], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 16:00:00')
#   }, index=[pd.to_datetime('2019-11-17 16:00')]),
#                        pd.DataFrame({
#     'vehicle count': [857], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 17:00:00')
#   }, index=[pd.to_datetime('2019-11-17 17:00')]),
#                          pd.DataFrame({
#     'vehicle count': [867], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 18:00:00')
#   }, index=[pd.to_datetime('2019-11-17 18:00')]),
#                             pd.DataFrame({
#     'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 01:00:00')
#   }, index=[pd.to_datetime('2020-01-17 01:00:00')]),
#                             pd.DataFrame({
#     'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 03:00:00')
#   }, index=[pd.to_datetime('2020-01-17 03:00:00')]),
#                           pd.DataFrame({
#     'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 02:00:00')
#   }, index=[pd.to_datetime('2020-01-17 02:00:00')])

  
# ])

In [430]:
data_df.loc[data_df.index == pd.to_datetime("2020-01-17 03:00:00"), "timestamp"] = pd.to_datetime("2020-01-17 03:00:00")

In [431]:
data_df[data_df.index >= pd.to_datetime("2020-01-17 00:00")].head(24)


,timestamp,vehicle count
DateTime,,


In [432]:
data_df = data_df[~(data_df.index.duplicated())]

In [433]:
data_df

,timestamp,vehicle count
DateTime,,
2015-11-01 00:00:00,2015-11-01 00:00:00,67
2015-11-01 01:00:00,2015-11-01 01:00:00,66
2015-11-01 02:00:00,2015-11-01 02:00:00,64
2015-11-01 03:00:00,2015-11-01 03:00:00,61
2015-11-01 04:00:00,2015-11-01 04:00:00,62
...,...,...
2016-12-21 11:00:00,2016-12-21 11:00:00,131
2016-12-21 12:00:00,2016-12-21 12:00:00,137
2016-12-21 13:00:00,2016-12-21 13:00:00,129


In [434]:
data_df = data_df.sort_index()

In [435]:
# data_df["vehicle count"] = np.where(data_df["timestamp"].dt.hour == 10, (data_df["vehicle count"] + 200), data_df["vehicle count"])


In [436]:
data_df = data_df[(data_df.index >= pd.to_datetime('2016-11-01')) & (data_df.index <= pd.to_datetime('2016-12-19 05:00'))]


In [437]:
data_df

,timestamp,vehicle count
DateTime,,
2016-11-01 00:00:00,2016-11-01 00:00:00,113
2016-11-01 01:00:00,2016-11-01 01:00:00,104
2016-11-01 02:00:00,2016-11-01 02:00:00,101
2016-11-01 03:00:00,2016-11-01 03:00:00,99
2016-11-01 04:00:00,2016-11-01 04:00:00,89
...,...,...
2016-12-19 01:00:00,2016-12-19 01:00:00,91
2016-12-19 02:00:00,2016-12-19 02:00:00,87
2016-12-19 03:00:00,2016-12-19 03:00:00,86


In [438]:
my_range_in1m = pd.date_range(start=data_df.index.min(), end=data_df.index.max(), freq='2H')


/var/folders/xk/ct1qcm4j5ljdj441cx4jphvjp9l86x/T/ipykernel_52852/2048691096.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  my_range_in1m = pd.date_range(start=data_df.index.min(), end=data_df.index.max(), freq='2H')


In [439]:
my_range_in1m[~my_range_in1m.isin(data_df.index)]


DatetimeIndex([], dtype='datetime64[ns]', freq='2h')

In [440]:
data_df1 = data_df
data_df1['timestamp1'] = data_df["timestamp"]


In [441]:
data_df = data_df.drop(columns=["timestamp1"])

In [442]:
data_df2 = data_df.groupby(pd.Grouper(freq='2H', key='timestamp')).sum()


/var/folders/xk/ct1qcm4j5ljdj441cx4jphvjp9l86x/T/ipykernel_52852/3828477597.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data_df2 = data_df.groupby(pd.Grouper(freq='2H', key='timestamp')).sum()


In [443]:
data_df2['timestamp'] = data_df2.index
data_df2

,vehicle count,timestamp
timestamp,,
2016-11-01 00:00:00,217,2016-11-01 00:00:00
2016-11-01 02:00:00,200,2016-11-01 02:00:00
2016-11-01 04:00:00,174,2016-11-01 04:00:00
2016-11-01 06:00:00,185,2016-11-01 06:00:00
2016-11-01 08:00:00,198,2016-11-01 08:00:00
...,...,...
2016-12-18 20:00:00,212,2016-12-18 20:00:00
2016-12-18 22:00:00,210,2016-12-18 22:00:00
2016-12-19 00:00:00,189,2016-12-19 00:00:00


In [444]:
data_df2["vehicle count"] = np.where(data_df2["vehicle count"] > 262, 262, data_df2["vehicle count"]) 

In [445]:
data_df2.to_json('./src/api/data/traffic_junction_simulated_15____091.json', orient="records")